In [1]:
#load packages
import os
import numpy as np
import pandas as pd
import requests 
import itertools

from datetime import date
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams

import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA 
from statsmodels.tsa.arima_model import ARMAResults 
from statsmodels.tsa.api import VAR 
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.stattools import adfuller, kpss
#from skopt import BayesSearchCV

sns.set_style('white')
sns.set_theme(style='white', palette='Paired')
pd.plotting.register_matplotlib_converters()
%matplotlib inline 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

#import warnings
#warnings.filterwarnings('ignore')

In [2]:
# Read in the dataset
folder_dir = '/kaggle/input/store-sales-time-series-forecasting'
df_event = pd.read_csv(os.path.join(folder_dir,'holidays_events.csv'))
df_oil = pd.read_csv(os.path.join(folder_dir,'oil.csv'))
df_stores = pd.read_csv(os.path.join(folder_dir,'stores.csv'))
df_train = pd.read_csv(os.path.join(folder_dir,'train.csv'))
df_test = pd.read_csv(os.path.join(folder_dir,'test.csv'))
df_transactions = pd.read_csv(os.path.join(folder_dir,'transactions.csv'))

### Exploratory Data Analysis

- There are 3,000,888 rows in training set.
- For a given store on a given day, there're 33 types of products (family)
- 54 unique stores, each store has 55,572 records of data.
    - Among that 55,572 records of data, there are 1684 unique dates, and on each day there're sales data for each of the 33 types of product (so 55,572 = 1684 * 33)

In [3]:
df_train.shape

(3000888, 6)

In [4]:
df_train.tail()

,id,date,store_nbr,family,sales,onpromotion
3000883,3000883,2017-08-15,9,POULTRY,438.13,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.55,1
3000885,3000885,2017-08-15,9,PRODUCE,"2,419.73",148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.00,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.00,0


In [5]:
df_train.describe()

,id,store_nbr,sales,onpromotion
count,"3,000,888.00","3,000,888.00","3,000,888.00","3,000,888.00"
mean,"1,500,443.50",27.50,357.78,2.60
std,"866,281.89",15.59,"1,102.00",12.22
min,0.00,1.00,0.00,0.00
25%,"750,221.75",14.00,0.00,0.00
50%,"1,500,443.50",27.50,11.00,0.00
75%,"2,250,665.25",41.00,195.85,0.00
max,"3,000,887.00",54.00,"124,717.00",741.00


Question: onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.
But how can we know the total number of items of a product family? Will it be more comparative if we know the percentage of items that are on promotion?

In [6]:
# Good: no missing data :)
missing = df_train.isnull().sum()
missing.sort_values(ascending = False)

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

#### EDA for df_stores:

In [7]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [8]:
df_stores.shape

(54, 5)

In [9]:
# Check for missing data
df_stores.isnull().sum() # No missing data
df_event.isnull().sum() # No missing data
df_test.isnull().sum() # No missing data
df_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

There are 43 dates that the oil price (dcoilwtico) is missing, we can impute the missing values by using the average of the price on previous day and the next day.

### Data Preprocessing

1. change date type
2. holiday event table: remove transferred; "what is bridge?"
3. merge all tables 
4. drop outliers
5. add an indicator for the payroll days (on the 15th and the last day of each month)
6. add an indicator for the earthquake on 2016-04-16 "greatly affect sales for few weeks after" 

Others:
1. Missing oil price imputation
2. Data encoding, data scaling (scale the numeric variables? standardization?)

##### 1. Change Data Type for date

In [10]:
for df in [df_event, df_oil, df_train, df_test, df_transactions]:
    df["date"] = pd.to_datetime(df['date'])

In [11]:
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0


In [12]:
d = df_train.set_index("date").groupby("store_nbr").resample("D").sales.sum().reset_index()
px.line(d, x = "date", y= "sales", color = "store_nbr", title = "Daily Total Sales By Store Number")

##### 2. Missing Data Imputation

Impute the missing oil price with the average of the previous and next day's oil price.

In [13]:
df_oil = df_oil.sort_values('date')

# Replace the rest missing value with AVG(prev_day + next_Day)
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].fillna((df_oil['dcoilwtico'].shift() + df_oil['dcoilwtico'].shift(-1)) / 2)

# Replace missing values with the value of the previous day if the next day is also missing
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].fillna(method='ffill')

# Replace missing values with the value of the next day if the previous day is also missing
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].fillna(method='bfill')

# Check if df_oil still has missing value
df_oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

In [14]:
df_event['type'].value_counts()

type
Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: count, dtype: int64

In [15]:
df_event = df_event.loc[df_event['transferred']==False].copy() 
df_event = df_event.loc[df_event['type'].isin(['Holiday','Additional','Bridge','Transfer'])].copy() 
df_event['holiday'] = 1

# NEXT STEP: ONE-HOT ENCODE THE different types of event
# e.g. event_holiday, event_additional, event_bridge, event_transfer

In [16]:
df_event_city = df_event.loc[df_event['locale']=='Local'].copy() 
df_event_state = df_event.loc[df_event['locale']=='Regional'].copy() 
df_event_nation = df_event.loc[df_event['locale']=='National'].copy() 

df_event_city = df_event_city[['date','locale_name','holiday']]
df_event_city.rename(columns={'locale_name':'city', 'holiday':'holiday_city'},inplace=True)
df_event_city.drop_duplicates(inplace=True)

df_event_state = df_event_state[['date','locale_name','holiday']]
df_event_state.rename(columns={'locale_name':'state', 'holiday':'holiday_state'},inplace=True)
df_event_state.drop_duplicates(inplace=True)

df_event_nation = df_event_nation[['date','holiday']]
df_event_nation.rename(columns={'holiday':'holiday_nation'},inplace=True)
df_event_nation.drop_duplicates(inplace=True)

In [17]:
df_train_m = pd.merge(df_train, df_transactions, how = 'left', on = ['date','store_nbr'])
df_train_m = pd.merge(df_train_m, df_oil, how = 'left', on = 'date')
df_train_m = pd.merge(df_train_m, df_stores, how = 'left', on = 'store_nbr')

df_train_m = pd.merge(df_train_m, df_event_city, how = 'left', on = ['date', 'city'])
df_train_m = pd.merge(df_train_m, df_event_state, how = 'left', on = ['date', 'state'])
df_train_m = pd.merge(df_train_m, df_event_nation, how = 'left', on = 'date')

In [18]:
df_train_m['holiday_flag'] = df_train_m['holiday_city'] + df_train_m['holiday_state'] + df_train_m['holiday_nation']
df_train_m['holiday'] = 0
df_train_m.loc[df_train_m['holiday_flag']>=1, 'holiday'] = 1 

##### 5. Adding indicator for payroll day

In [19]:
df_train['is_month_end'] = df_train.date.dt.is_month_end.astype("int8")
df_train['day_of_month'] = df_train.date.dt.day.astype("int8")
df_train["payroll_day"] = pd.Series(np.where((df_train['is_month_end'] == 1) | (df_train["day_of_month"] == 15), 1, 0)).astype("int8")

### Other Potential Feature Engineering

- DayOfWeek
- MonthOfYear
- Group-level features: cluster (grouping of similar stores)
- Lag Feature

In [20]:
# Day-of-week
df_train['day_of_week'] = (df_train.date.dt.dayofweek + 1).astype("int8")
# Month-of-year
df_train['month'] = df_train.date.dt.month.astype("int8")

# Season: Winter:0, Spring:1, Summer:2, Fall:3
df_train["season"] = np.where(df_train.month.isin([12,1,2]), 0, 1)
df_train["season"] = np.where(df_train.month.isin([6,7,8]), 2, df_train["season"])
df_train["season"] = pd.Series(np.where(df_train.month.isin([9, 10, 11]), 3, df_train["season"])).astype("int8")

In [21]:
df_train.tail()

,id,date,store_nbr,family,sales,onpromotion,is_month_end,day_of_month,payroll_day,day_of_week,month,season
3000883,3000883,2017-08-15,9,POULTRY,438.13,0,0,15,1,2,8,2
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.55,1,0,15,1,2,8,2
3000885,3000885,2017-08-15,9,PRODUCE,"2,419.73",148,0,15,1,2,8,2
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.00,8,0,15,1,2,8,2
3000887,3000887,2017-08-15,9,SEAFOOD,16.00,0,0,15,1,2,8,2
